# Tema 2 - exercitii NumPy

Se acordă doua puncte din oficiu. Fișierul va fi denumit tema2_ia_id_nume_prenume.ipynb. Verificați înainte de trimitere faptul ca execuția celulelor de sus în jos funcționează corespunzător. Aserțiunile sunt obligatoriu a fi indeplinite. Suplimentar, puteti face alte verificari.

Se vor folosi type annotations pentru variabile, parametrii tuturor funcțiilor, tipuri de retur. Se vor folosi docstrings pentru toate funcțiile. Neîndeplinirea acestei cerințe duce la înjumătățirea punctajului pentru exercițiul în cauză.  

Tema va fi predata pana cel tarziu 23.11.2024, pe platforma.

1. (1p) Pentru un vector de numere, care sunt toate pozitiile pe care apare valoarea maxima?
Folositi functii NumPy.

In [1]:
from typing import Tuple
import numpy as np
v: np.array = np.array([1, 2, 3, 4, 4, 3, 1] * 2, dtype=np.float32)  # de adaugat type annotations

# renamed variable from _min to _max
positions_max: np.array = np.array(np.where(v == np.max(v)))

assert np.all(positions_max == [3, 4, 10, 11])

2. (1p) Construiti o functie care, primind o matrice, determina pe ce pozitii (linii, coloane) se afla valorile in intervalele [a, b] sau [c, d] date ca parametri. Functia va returna un tuplu de doi vectori: primul e cu indicii de linie, al doilea cu indicii de coloana unde se afla elementele cautate.

Exemplu:

In [2]:
def pos_values(matNxN: np.ndarray, a: int, b: int, c: int, d: int) -> Tuple[np.array, np.array]:  # de adaugat type annotations pentru parametri si tip de return
    ''' 
    Returneaza un tuplu de doi vectori, primul contine indicii liniilor, al doilea contine indicii coloanelor unde se afla elementele cautate
    '''

    rows: np.array = []
    cols: np.array = []
    rows, cols = np.where((matNxN >= a) & (matNxN <= b) | (matNxN >= c) & (matNxN <= d))            
    return rows, cols
    

mat = np.arange(12).reshape(3, 4)
rows, cols = pos_values(mat, 2, 4, 6, 8)
assert np.all(rows == [0, 0, 1, 1, 1, 2])
assert np.all(cols == [2, 3, 0, 2, 3, 0])

mat = -np.arange(12).reshape(3, 4)
rows, cols = pos_values(mat, 2, 4, 6, 8)
assert np.all(rows == [])
assert np.all(cols == [])

3. (1p) Se da un vector cu numere intregi nenule. Sa se determine toti indicii `i` unde diferenta `v[i]-v[i-1] > p`, cu `p` prag dat. Codul se va scrie intr-o functie adnotata.

Exemplu: 
```python
v = np.array([1, 2, 13, 3, 13, 200, 300, -10, 10, 100])
p = 5
```
Trebuie sa rezulte vectorul de indici: `[2, 4, 5, 6, 8, 9]`.

De luat in considerare ca functia `np.where` gaseste indicii unde anumite proprietati sunt indeplinite.

In [3]:
def jumping(v: np.array, p: int) -> np.array:  # de adnotat, doctstring
    assert np.all( v != 0), 'All values should be non-zero'
    '''
    Determina toti indicii i pentru care v[i] - v[i-1] > p 
    '''
    return np.where(v[1:] - v[:-1] > p)[0] + 1

# test 1
v = np.array([1, 2, 13, 3, 13, 200, 300, -10, 10, 100])
p = 5
assert np.all(jumping(v, p) == [2, 4, 5, 6, 8, 9])

# test 2: random vector, naive counting
v = np.random.randint(-100, 100, 1000)
# naive way of counting jumps:
positions = []
for i in range(1, len(v)):
    if v[i] - v[i-1] > p:
        positions.append(i)

v[np.where(v == 0)] = 1 # v values are generated randomly, so there might be zeros. Need this to avoit assertion
assert np.all(positions == jumping(v, p))

4. (1p) Generati o matrice aleatoare de dimensiune m*n care are m valori nan (not-an-number, https://numpy.org/doc/stable/reference/constants.html), la pozitii aleatoare, cate o pozitie pe fiecare linie. Determinati apoi suma elementelor non-nan din matrice in doua feluri (construiti doua functii). Comparati sumele returnate.

Exemplu:
```python
a = get_matrix(3, 4)
# sa presupunem ca a este
a = 
    1.5 2.2 np.nan
    4.3 np.nan 1.0
    10.1 np.nan -1.1
```

Suma elementelor este 19.0.
Indicatii: folositi functia `np.random.choice` cu parametri adecvati pentru a alege indici in mod aleator, fara repetare. 

In [4]:
def get_matrix(m: int, n: int) -> np.ndarray:  # de adnotat, docstring
    '''
    Genereaza o matrice de dimensiune m x n cu valori aleatoare, cu exceptia a m valori nan
    '''
    a = np.random.rand(m, n)
    a[np.random.choice(m, m), np.random.choice(n, m)] = np.nan
    assert np.sum(np.isnan(a)) == m  # verificare: avem exact m nan-uri
    return a

def sum_1(a: np.ndarray) -> float: # de adnotat, docstring
    '''
    Calculeaza suma elementerlor non-nan dintr-o matrice
    '''
    return np.nansum(a)


def sum_2(a: np.ndarray) -> float: # de adnotat, docstring
    ''' 
    Calculeaza suma elementerlor non-nan dintr-o matrice
    '''
    return np.sum(a[~np.isnan(a)])

def naive_sum(a: np.ndarray) -> float:
    '''
    Calculeaza suma elementelor non-nan dintr-o matrice
    '''
    s = 0
    m, n = a.shape
    for i in range(m):
        for j in range(n):
            s += a[i, j] if not np.isnan(a[i, j]) else 0
    return s

m, n = 3, 4
a = get_matrix(m, n)

assert np.allclose(sum_1(a), naive_sum(a))
assert np.allclose(sum_2(a), naive_sum(a))

# intrebare petru studenti: de ce e nevoie de allclose si nu se face comparatie cu == ?
# Raspuns: deoarece valorile float sunt reprezentate in memorie cu o anumita precizie, iar operatiile cu float-uri pot introduce erori de rotunjire.

# Returns True if two arrays are element-wise equal within a tolerance.
# The tolerance values are positive, typically very small numbers. 
# The relative difference (rtol * abs(b)) and the absolute difference atol 
# are added together to compare against the absolute difference between a and b.

# Examples
# >>> import numpy as np
# >>> np.allclose([1e10,1e-7], [1.00001e10,1e-8])
# False
# >>> np.allclose([1e10,1e-8], [1.00001e10,1e-9])
# True
# >>> np.allclose([1e10,1e-8], [1.0001e10,1e-9])
# False
# >>> np.allclose([1.0, np.nan], [1.0, np.nan])
# False
# >>> np.allclose([1.0, np.nan], [1.0, np.nan], equal_nan=True)
# True

5. (1p) Folosind functia `np.convolve`, sa se determine media alunecatoare (moving average) pentru un vector de numere; lungimea ferestrei alunecatore se da ca parametru al functiei construite de voi. Comparati rezultatele si viteza de executie cu a functiaie `naive_moving_average`.

Referinte:
1. [Moving average](https://en.wikipedia.org/wiki/Moving_average)
1. [np.convolve: How to Use Numpy convolve() Method](https://appdividend.com/2022/10/14/np-convolve/)

In [5]:
def moving_average_fast(x: int, w: int) -> np.ndarray:  # de adnotat, docstring
    ''' 
    Calculeaza moving average-ul a unui vector x cu fereastra de lungime w
    '''

    assert x.ndim == 1, 'One dimension numpy array'
    assert w > 1, 'Average of at least two values'
    return np.convolve(x, np.ones(w), 'valid') / w 

def moving_average_slow(x: int, w: int) -> np.ndarray: # de adnotat, docstring
    ''' 
    Calculeaza media mobila a unui vector x cu fereastra de lungime w
    '''
    assert x.ndim == 1, 'One dimension numpy array'
    assert w > 1, 'Average of at least two values'
    result = np.zeros(x.shape[0]-w+1)
    for i in range(x.shape[0]-w+1):
        result[i] = np.mean(x[i:i+w])
    return result

len_vec, w = 10000, 10
x = np.random.rand(len_vec)

a = moving_average_fast(x, w)
b = moving_average_slow(x, w)

assert len(a) == len(b), 'The two methods should return vectors of the same lengths'
assert np.allclose(a, b), 'The two methods should return close vectors'

%timeit moving_average_fast(x, w)
%timeit moving_average_slow(x, w)

51.6 μs ± 8.4 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
67.4 ms ± 7.94 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


6. (1p) Se dau vectorii de valori $x=[x_0 \dots x_{n-1}]$, $t=[t_0, \dots t_n-1]$, cu $t[0] < t[1] < \dots$. Sa se calculeze intr-o maniera vectorizata vectorul $v$ ale carui componente sunt:
$$
v[i] = \frac{\sum\limits_{k=0}^{i+1} x_k}{t[i+1] - t[i]}
$$

Comparati rezultatele si viteza de executie cu o implementare care construieste vectorul $v$ pas cu pas.

Optional: faceti o implementare folosind Numba. 

In [6]:
import numpy as np

n = 10000
x = np.random.rand(n)
t = np.random.randint(1, 10, n)
t = t.cumsum()

def is_increasing(t: np.array) -> bool: 
    ''' 
    Verifica daca un vector este crescator
    '''
    return np.all(t[1:] >= t[:-1])

def fast_v(x: np.array, t: np.array) -> np.array: # de adnotat, docstring
    assert is_increasing(t), 'non increasing values'
    ''' 
    Interpoleaza un vector x la momentele de timp din t
    '''
    return np.interp(np.arange(t[-1]+1), t, x)

def slow_v(x: np.array, t: np.array) -> np.array: # de adnotat, docstring
    assert is_increasing(t), 'non increasing values'
    '''
    Interpoleaza un vector x la momentele de timp din t
    '''

    return np.array([np.interp(i, t, x) for i in range(t[-1]+1)])

v1 = fast_v(x, t)
v2 = slow_v(x, t)

assert v1.shape == v2.shape
assert np.allclose(v1, v2)

%timeit fast_v(x, t)
%timeit slow_v(x, t)

660 μs ± 135 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
487 ms ± 81.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


7. (2p) Se da o matrice $a$ cu numere reale. 
    1. Sa se calculeze o alta matrice $b$ care are valorile calculate astfel:
    $$
    b[i, j] = \frac{a[i, j] - MIN[j]}{MAX[j]-MIN[j]}
    $$
    unde $MAX[j]$ si $MIN[j]$ sunt valorile minime, respectiv maxime din coloana $j$ a matricei $a$. Tranformarea se face printr-o functie care preia o matrice si returneaza un tuplu compus dintr-o matrice $b$ si vectorii $MIN$ si $MAX$. Daca o coloana este constanta, valorile ei se lasa nemodificate. 
    2. Verificati ca in urma acestei transformari matricea $b$ are valorile in intervalul $(-epsilon, 1+epsilon)$, unde $epsilon=1e-5$, exceptand coloanele constante.

In [7]:
from typing import Tuple

import numpy as np
m, n = 100, 200

a = np.random.randn(m, n)

# rezolvare pct A
def min_max_scale(x: np.ndarray): # de adnotat, docstring
    '''
    Genereaza o alta matrice care are valorile calculate dupa formula (x[i, j] - min(x)) / (max(x) - min(x))
    Returneaza matricea scalata, valoarea minima si valoarea maxima
    '''
    b = [ (x[i, j] - np.min(x)) / (np.max(x) - np.min(x)) for i in range(x.shape[0]) for j in range(x.shape[1])]

    return b, np.min(b), np.max(b)

# rezolvare pct B
def check_scaling(y: np.ndarray, eps = 1e-5): # de adnotat, docstring
    assert eps > 0
    ''' 
    Verifica daca valorile sunt scalate se afla in intervalul (-eps, 1+eps)
    '''
    return np.abs(np.min(y) - 0) < eps and np.abs(np.max(y) - 1) < eps

assert check_scaling(min_max_scale(a)[0])
